In [8]:
import tensorflow as tf
import numpy as np
import PIL.Image as Image
import xNN
xNN.load_ito_ops()

In [7]:
# 用pb做inference
def inference(jpg_path, pb_file_path):
    with tf.Graph().as_default():
        output_graph_def = tf.GraphDef()

        with open(pb_file_path, "rb") as f:
            output_graph_def.ParseFromString(f.read())
            _ = tf.import_graph_def(output_graph_def, name="")

        with tf.Session() as sess:
            init = tf.global_variables_initializer()
            sess.run(init)

            input_x = sess.graph.get_tensor_by_name("input/placeholder:0")
            print(input_x)
            out_softmax = sess.graph.get_tensor_by_name("MobilenetV3/Logits/output:0")
            print(out_softmax)

            img = Image.open(jpg_path)
            print(img.size)
            img = img.resize((224, 224))
            img = np.array(img)
            print(img.shape)
            img = (img / 127.5) - 1.0
            img_out_softmax = sess.run(out_softmax, feed_dict={input_x:np.reshape(img, [-1, 224, 224, 3])})

            print("img_out_softmax:",img_out_softmax)
            prediction_labels = np.argmax(img_out_softmax, axis=1)
            print("label:",prediction_labels)

# inference("bus.jpg", "AUTO-inference.pb")

In [56]:

from tensorflow.python import pywrap_tensorflow

# 从ckpt中读取信息
ckpt_path = 'model_final.ckpt'
reader = pywrap_tensorflow.NewCheckpointReader(ckpt_path)
reader.get_tensor
var_names = [name for name in reader.get_variable_to_shape_map().keys()]
var_names_deri = [name for name in var_names if 'expanded_conv_9/expand' in name]
print(var_names_deri)
var_name_desi = 'MobilenetV3/expanded_conv_9/expand/weights'
tensor = reader.get_tensor(var_name_desi)
mean = reader.get_tensor('MobilenetV3/expanded_conv_9/expand/BatchNorm/moving_mean')
vari = reader.get_tensor('MobilenetV3/expanded_conv_9/expand/BatchNorm/moving_variance')
gamma = reader.get_tensor('MobilenetV3/expanded_conv_9/expand/BatchNorm/gamma')
beta = reader.get_tensor('MobilenetV3/expanded_conv_9/expand/BatchNorm/beta')
tensor = tensor.transpose(0,1,3,2)
tensor = tensor.squeeze(0).squeeze(0)
print(mean[0])
print(vari[0])
print(gamma[0])
print(beta[0])
# print(tensor.shape)
print(tensor[0])
fuse = ((tensor - np.expand_dims(mean, axis=1))/np.expand_dims(np.sqrt(vari), axis=1))*np.expand_dims(beta, axis=1)
print(fuse[0])
# +np.expand_dims(gamma, axis=1)

['MobilenetV3/expanded_conv_9/expand/weights/weight_optimize/ito/quant/max', 'MobilenetV3/expanded_conv_9/expand/weights/weight_optimize/ito/quant/absmax_ema', 'MobilenetV3/expanded_conv_9/expand/BatchNorm/gamma', 'MobilenetV3/expanded_conv_9/expand/weights/weight_optimize/ito/prune/update_mask', 'MobilenetV3/expanded_conv_9/expand/weights/weight_optimize/ito/step', 'MobilenetV3/expanded_conv_9/expand/weights/weight_optimize/ito/quant/min', 'MobilenetV3/expanded_conv_9/expand/BatchNorm/beta', 'MobilenetV3/expanded_conv_9/expand/BatchNorm/moving_mean', 'MobilenetV3/expanded_conv_9/expand/BatchNorm/moving_variance', 'MobilenetV3/expanded_conv_9/expand/weights', 'MobilenetV3/expanded_conv_9/expand/weights/weight_optimize/ito/prune/mask', 'MobilenetV3/expanded_conv_9/expand/weights/weight_optimize/ito/prune/prune_percent']
-0.01858414
29.914358
1.4044236
1.4314002
[-5.963e-02  1.154e-01 -9.664e-02 -1.232e-01  4.882e-02 -1.454e-01
  1.129e-01  6.263e-02  9.434e-02  1.079e-01 -1.384e-01  5.0

In [38]:
from tensorflow.python.platform import gfile

sess = tf.Session()
with gfile.FastGFile('AUTO-inference.pb', 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    sess.graph.as_default()
    tf.import_graph_def(graph_def, name='') # 导入计算图

# 需要有一个初始化的过程    
sess.run(tf.global_variables_initializer())
bn_fuse_name = sess.graph.get_tensor_by_name('MobilenetV3/expanded_conv_9/expand/mul_fold:0')
print(bn_fuse_name.shape)
val = sess.run(bn_fuse_name).transpose(0,1,3,2)
print(val[0][0][0])

(1, 1, 96, 576)
[-1.531e-02  2.963e-02 -2.481e-02 -3.162e-02  1.254e-02 -3.735e-02
  2.899e-02  1.608e-02  2.422e-02  2.772e-02 -3.553e-02  1.306e-02
  1.386e-02  5.992e-03 -1.966e-02  3.118e-02 -2.461e-02 -2.345e-02
 -6.373e-02 -2.178e-02  1.760e-02 -2.689e-03  3.306e-02 -1.134e-02
 -1.223e-02  3.243e-03 -6.257e-02 -5.088e-03  8.471e-03 -1.380e-02
 -3.736e-02  2.711e-02 -1.103e-02 -1.143e-02 -2.738e-02  2.451e-03
  8.213e-03 -2.791e-02 -4.600e-02  3.623e-02 -3.906e-02 -6.176e-03
  1.613e-02  2.354e-02 -1.019e-02  6.894e-03 -5.801e-03  2.570e-02
  9.833e-03  5.098e-02  3.908e-02 -2.981e-02 -2.249e-02  6.491e-02
  3.270e-02  6.183e-03  5.329e-02 -7.539e-03  2.160e-02  6.536e-05
 -5.304e-03 -3.322e-03  5.712e-02  2.504e-02 -3.396e-03  3.828e-02
 -3.293e-02  7.349e-03  2.600e-02 -4.021e-02  1.742e-02  1.083e-01
  1.446e-02  6.529e-02  2.752e-03  2.205e-02  3.699e-02 -1.104e-02
 -2.435e-02  1.914e-02 -4.126e-03  2.164e-02  1.069e-01  2.964e-02
 -2.231e-02 -8.928e-03 -7.499e-03 -6.284e-02 -